# Data + Python

Advanced methods for the desperate data practitioner 

In [39]:
# coopl thing -- "contextual help" in jlab

In [9]:
import buckaroo
from buckaroo.buckaroo_widget import BuckarooWidget # manually view
buckaroo.disable() # multi-indexes don't show


## Pandas


In [36]:
import pandas as pd

df = pd.read_parquet("companies.parquet")

In [10]:
df

,Unnamed:_0,name,domain,year_founded,industry,size_range,locality,country,linkedin_url,current_employee_estimate,total_employee_estimate,tld
0,5872184,ibm,ibm.com,1911,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906,com
1,4425416,tata consultancy services,tcs.com,1968,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369,com
2,21074,accenture,accenture.com,1989,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768,com
3,2309813,us army,goarmy.com,1800,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958,com
4,1558607,ey,ey.com,1989,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960,com
...,...,...,...,...,...,...,...,...,...,...,...,...
7173404,5101935,squad sp. z o. o.,squad.com.pl,2013,internet,1 - 10,"litzmannstadt, lodzkie, poland",poland,linkedin.com/company/squad-sp.-z-o.-o.,0,1,com
7173411,4768458,"fuller, hadeed, & ros-planas, pllc",fhrlaw.com,2015,law practice,1 - 10,"virginia beach, virginia, united states",united states,linkedin.com/company/fhrlaw,0,2,com
7173416,3515286,fit plus s.r.o.,fitplus.sk,1993,"health, wellness and fitness",1 - 10,"bratislava, bratislavský, slovakia",slovakia,linkedin.com/company/fit-plus-s.r.o.,0,1,sk
7173417,5481997,coriex srl,coriex.it,1987,apparel & fashion,1 - 10,"padova, veneto, italy",italy,linkedin.com/company/coriex-srl,0,4,it


In [37]:
df.columns = df.columns.str.replace(" ", "_")
# Ensure no null values are present in `year_founded` column
df = df[df.year_founded.notnull() & df.country.notnull()]
# Cast year_founded as int
df.year_founded = df.year_founded.astype(int)
# Extract top-level domain
df["tld"]=df.domain.str.rsplit(".").str[1]

In [38]:
df.tld.value_counts()

tld
com         1662048
co           167584
org          138624
nl            82495
net           64933
             ...   
trentino          1
idrett            1
komatsu           1
iwi               1
zt                1
Name: count, Length: 951, dtype: int64

In [15]:
df.groupby(by=["tld","country"]).domain.count()

tld      country       
a        sweden              1
ab       canada            129
         united kingdom      1
abbott   united states       1
abogado  spain               2
                          ... 
zp       italy               1
         ukraine             5
zt       ukraine             1
рф       russia              4
укр      ukraine             1
Name: domain, Length: 11945, dtype: int64

In [41]:
# for later
pandas_df = df.copy()

## Polars

The goal of Polars is to provide a lightning fast DataFrame library that:

* Utilizes all available cores on your machine.
* Optimizes queries to reduce unneeded work/memory allocations.
* Handles datasets much larger than your available RAM.
* Has an API that is consistent and predictable.
* Has a strict schema (data-types should be known before running the query).
* Polars is written in Rust which gives it C/C++ performance and allows it to fully control performance critical parts in a query engine.

As such Polars goes to great lengths to:

* Reduce redundant copies.
* Traverse memory cache efficiently.
* Minimize contention in parallelism.
* Process data in chunks.
* Reuse memory allocations.

In [42]:
import polars as pl

df = pl.scan_parquet("companies.parquet")
df = df.rename(mapping={col:col.replace(" ", "_") for col in df.columns})
# Cast year_founded as int
df = (
    df.with_columns(pl.col("year_founded").cast(pl.Int32, strict=True))
    .rename({"Unnamed:_0": "company_id"})
    .filter(pl.col("country").is_not_null() & pl.col("year_founded").is_not_null())
)
df = (
    df.with_columns(pl.col("domain").str.split(".").list.last().alias("tld"))
)

In [43]:
df.explain()

' WITH_COLUMNS:\n [col("domain").str.split([Utf8(.)]).list.get([-1]).alias("tld")]\n  RENAME\n    DROP_NULLS by: [country, year_founded]\n       WITH_COLUMNS:\n       [col("year_founded").strict_cast(Int32)]\n        RENAME\n\n            Parquet SCAN companies.parquet\n            PROJECT */11 COLUMNS'

In [19]:
df.head().collect()

company_id,name,domain,year_founded,industry,size_range,locality,country,linkedin_url,current_employee_estimate,total_employee_estimate,tld
i64,str,str,i32,str,str,str,str,str,i64,i64,str
5872184,"""ibm""","""ibm.com""",1911,"""information te…","""10001+""","""new york, new …","""united states""","""linkedin.com/c…",274047,716906,"""com"""
4425416,"""tata consultan…","""tcs.com""",1968,"""information te…","""10001+""","""bombay, mahara…","""india""","""linkedin.com/c…",190771,341369,"""com"""
21074,"""accenture""","""accenture.com""",1989,"""information te…","""10001+""","""dublin, dublin…","""ireland""","""linkedin.com/c…",190689,455768,"""com"""
2309813,"""us army""","""goarmy.com""",1800,"""military""","""10001+""","""alexandria, vi…","""united states""","""linkedin.com/c…",162163,445958,"""com"""
1558607,"""ey""","""ey.com""",1989,"""accounting""","""10001+""","""london, greate…","""united kingdom…","""linkedin.com/c…",158363,428960,"""com"""


In [32]:
df.group_by(by=["tld","country"]).agg(pl.col("domain").count()).collect()

tld,country,domain
str,str,u32
"""com""","""united kingdom…",93390
"""br""","""brazil""",61737
"""com""","""turkey""",16643
"""com""","""united arab em…",10683
"""com""","""italy""",23607
"""com""","""algeria""",655
"""com""","""qatar""",1025
"""com""","""singapore""",7137
"""com""","""hong kong""",3721


In [45]:
polars_df = df

## DuckDB

In [105]:
import duckdb 
db = duckdb.connect(":memory:")
db.sql("SELECT tld, country, count(domain) FROM pandas_df GROUP BY tld, country")


┌─────────┬────────────────────┬─────────────────┐
│   tld   │      country       │ count("domain") │
│ varchar │      varchar       │      int64      │
├─────────┼────────────────────┼─────────────────┤
│ in      │ portugal           │               8 │
│ it      │ italy              │           39994 │
│ dk      │ denmark            │           15191 │
│ com     │ czechia            │            1820 │
│ com     │ new zealand        │            3031 │
│ com     │ indonesia          │            6153 │
│ es      │ spain              │           24033 │
│ pt      │ portugal           │            8838 │
│ org     │ canada             │            3240 │
│ org     │ brazil             │            1633 │
│  ·      │   ·                │               · │
│  ·      │   ·                │               · │
│  ·      │   ·                │               · │
│ golf    │ netherlands        │               2 │
│ today   │ india              │               4 │
│ info    │ dominican republic 

In [104]:
db.sql("SELECT tld, country, count(domain) FROM polars_df GROUP BY tld, country")

┌─────────┬──────────────────────┬─────────────────┐
│   tld   │       country        │ count("domain") │
│ varchar │       varchar        │      int64      │
├─────────┼──────────────────────┼─────────────────┤
│ com     │ austria              │            2522 │
│ com     │ india                │           62357 │
│ se      │ sweden               │           19051 │
│ NULL    │ united states        │               0 │
│ com     │ peru                 │            3970 │
│ com     │ philippines          │            3339 │
│ fr      │ france               │           30203 │
│ io      │ estonia              │              66 │
│ org     │ spain                │            2440 │
│ de      │ germany              │           20972 │
│ ·       │    ·                 │               · │
│ ·       │    ·                 │               · │
│ ·       │    ·                 │               · │
│ xxx     │ czechia              │               1 │
│ ai      │ hungary              │            

### Natively

In [133]:
companies = db.sql("SELECT * FROM companies.parquet WHERE country is not NULL")
# or
companies = db.read_parquet("companies.parquet").filter("country is not NULL")

# Obtain column names and replace spaces with underscores
names = companies.columns
names = [(name, name.replace(" ", "_").lower()) for name in names]
# Rename first column to "company_id"
names[0] = ("Unnamed: 0", "company_id")
projection = ", ".join([f'"{old}" AS {new}' for old, new in names])
companies = companies.project(projection)

companies.sql_query()

'SELECT "Unnamed: 0" AS company_id, "name" AS "name", "domain" AS "domain", "year founded" AS year_founded, industry AS industry, "size range" AS size_range, locality AS locality, country AS country, "linkedin url" AS linkedin_url, "current employee estimate" AS current_employee_estimate, "total employee estimate" AS total_employee_estimate FROM (SELECT * FROM (SELECT * FROM parquet_scan(\'companies.parquet\', (union_by_name = false), (hive_partitioning = false), (filename = false), (file_row_number = false), (binary_as_string = false))) AS parquet_18a144e912448aa2 WHERE (country IS NOT NULL)) AS parquet_18a144e912448aa2'

In [134]:
# add our TLD column
#companies_tld = db.sql("SELECT split_part(domain, '.', -1) as TLD, * FROM companies")
companies = companies.query("self", "SELECT split_part(domain, '.', -1) as TLD, * FROM self")

In [135]:
db.sql("SELECT tld, domain, country, count(domain) FROM companies_tld GROUP BY tld, country, domain")


┌─────────┬────────────────────────────┬────────────────┬─────────────────┐
│   TLD   │           domain           │    country     │ count("domain") │
│ varchar │          varchar           │    varchar     │      int64      │
├─────────┼────────────────────────────┼────────────────┼─────────────────┤
│ uk      │ crossfitmci.co.uk          │ united kingdom │               1 │
│ com     │ acathospital.com           │ united states  │               1 │
│ uk      │ youngandtalented.co.uk     │ united kingdom │               1 │
│ de      │ macor.de                   │ germany        │               1 │
│ fi      │ tripod.fi                  │ finland        │               1 │
│ uk      │ edenshop.co.uk             │ united kingdom │               1 │
│ com     │ donlawleyco.com            │ united states  │               1 │
│ uk      │ justlistsapp.co.uk         │ united kingdom │               1 │
│ be      │ beschutwonenkempen.be      │ belgium        │               1 │
│ org     │ 